## PS4-4 Independent components analysis

#### (a) Gaussian source

From

\begin{align*}
\nabla_W \ell (W) & = \nabla_W \sum_{i = 1}^{m} \big( \log \vert W \vert + \sum_{j = 1}^{n} \log g' (w_j^T x^{(i)}) \big) \\
                  & = m(W^{-1})^T - \frac{1}{2} \sum_{i = 1}^{m} \nabla_W \sum_{j = 1}^{n} (w_j^T x^{(i)})^2 \\
                  & = m(W^{-1})^T - W X^T X \\
                  & = 0
\end{align*}

we have

$$W^T W = m (X^T X)^{-1}$$

Let R be an arbitrary orthogonal matrix, then if the data had been mixed according to $W' = RW$,

$$W'^T W' = (RW)^T (RW) = W^T R^T R W = W^T W = m (X^T X)^{-1}$$

Therefore, there is no  way to tell whether the sources were mixed using $W$ or $W'$.

#### (b) Laplace source

For a single example,

\begin{align*}
\nabla_W \ell (W) & = \nabla_W \log \vert W \vert + \sum_{j = 1}^{n} \log g' (w_j^T x^{(i)}) \\
                  & = (W^{-1})^T - \nabla_W \sum_{j = 1}^{n} \vert w_j^T x^{(i)} \vert \\
                  & = (W^T)^{-1} - \mathrm{sign} (W x^{(i)}) (x^{(i)})^T
\end{align*}

The update rule is

$$W := W + \alpha \big( (W^T)^{-1} - \mathrm{sign} (W x^{(i)}) (x^{(i)})^T \big)$$

#### (c) Cocktail Party Problem

In [1]:
import numpy as np
import scipy.io.wavfile

In [2]:
def update_W(W, x, learning_rate):
    """
    Perform a gradient ascent update on W using data element x and the provided learning rate.

    This function should return the updated W.

    Use the laplace distribution in this problem.

    Args:
        W: The W matrix for ICA
        x: A single data element
        learning_rate: The learning rate to use

    Returns:
        The updated W
    """

    # *** START CODE HERE ***

    updated_W = W + learning_rate * (np.linalg.inv(W.T) - np.outer(np.sign(W @ x), x.T))

    # *** END CODE HERE ***

    return updated_W

In [3]:
def unmix(X, W):
    """
    Unmix an X matrix according to W using ICA.

    Args:
        X: The data matrix
        W: The W for ICA

    Returns:
        A numpy array S containing the split data
    """

    S = np.zeros(X.shape)


    # *** START CODE HERE ***

    S = X @ W.T

    # *** END CODE HERE ***

    return S

Run ICA:

In [6]:
Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

def save_sound(audio, name):
    scipy.io.wavfile.write('output/{}.wav'.format(name), Fs, audio)

def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1 , 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01 , 0.01, 0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    for lr in anneal:
        print(lr)
        rand = np.random.permutation(range(M))
        for i in rand:
            x = X[i]
            W = update_W(W, x, lr)

    return W

X = normalize(load_data())

print(X.shape)

for i in range(X.shape[1]):
    save_sound(X[:, i], 'mixed_{}'.format(i))

W = unmixer(X)
print(W)
S = normalize(unmix(X, W))

for i in range(S.shape[1]):
    save_sound(S[:, i], 'split_{}'.format(i))

(53442, 5)
Separating tracks ...
0.1
0.1
0.1
0.05
0.05
0.05
0.02
0.02
0.01
0.01
0.005
0.005
0.002
0.002
0.001
0.001
[[ 52.84960754  16.80078835  19.94882758 -10.19906568 -20.90595341]
 [ -9.92376916  -0.97707867  -4.67376535   8.04189743   1.78878696]
 [  8.29991551  -7.47071876  19.29533856  15.18781696 -14.33002796]
 [-14.66771291 -26.64217528   2.4541469   21.37788898  -8.41621517]
 [ -0.27347678  18.38787404   9.30862356   9.11105981  30.59817989]]
